### This notebook was created by the tectonic geomorphology group at ESPIn 2021. In this tutorial, we will discuss and execute a model featured in Reitman et al. (2015) that simulated strike-slip faulting (SSFM). We will then demonstrate simple ways to expand upon the model's original functionality to simulate landscape evolution during oblique faulting and subsequent orographic precipitation. The model's code and the paper's simulation information can be found here: https://zenodo.org/record/3374026#.YMfJpTZudUN

### To begin, create and activate a conda environment with the following packages installed: numpy, matplotlib, scipy, scikit-image, scikit-learn, pickle, yaml, rasterio, landlab.

### SSFM calls several original functions, which we will need to define before running the model. In the online repository, these functions are located in 'slip.py'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['axes.facecolor'] =(1,1,1,1)  # white plot backgrounds
plt.rcParams['figure.facecolor']=(1,1,1,0) # clear figure backgrounds

###-------------------------------------------------------------------------###
#%% define functions
###-------------------------------------------------------------------------###

def calcOFD(fault_zone_width,max_slip,fault_loc,y_star,dxy,ymax,nrows,plot=False,save_plot=False):
    y = np.linspace(0,ymax,nrows) # zone to plot on y/grid axis
    b = 0. # center on this point on x/slip axis
    a = max_slip/2. # this divides slip between both sides of the fault.
    y_zero = fault_loc # center on this point on y/grid axis
    y_star = y_star # length scale over which OFD decay occurs
    slip = b + a*np.tanh((y-y_zero)/y_star) # calculate ofd profile with hyperbolic tangent

    if plot:
        plt.plot(slip,y,color='navy')
        plt.axhline(fault_loc,0,1,color='k',linestyle='--',linewidth=1,)
        plt.grid(color='lightgray',linestyle='--')
        plt.ylabel('distance (m)')
        plt.xlabel('slip (m)')
        plt.title('Strike-Slip Displacement Profile')
        plt.tight_layout()
        if save_plot:
            plt.savefig('ofd_profile.png', dpi=500,transparent=True)
        plt.show()

    return slip

def calcSlipRegime (tmax, dt, interval, total_slip=0, slip_rate=0, slip_type='characteristic',std=1,seed=None,plot=False):
    '''function to make an array (size = tmax/dt) of slip values based on different earthquake recurrence models.
    slip_type can be 'characteristic', 'random_time', 'random_slip', 'random_random', or 'creep'.
    defaults to characteristic.
    WARNING: random options are not physics-based (i.e., do not depend on previous earthquake time or size)
        characteristic = constant time and slip  - same time interval between earthquakes of equal magnitude until reach total_slip
        creep = characteristic with slip every dt
        random_time: random time and predictable slip - random time interval between earthquakes of equal magnitude - sums to total_slip
        random_slip: predictable time and random slip - same time interval between earthquakes of random magnitude - sums to total_slip
        random_random: random time intervals and random slip amount - sums to total_slip
        random_std: random sampling of normal distributions with centers at interval and std=std. interval and std should be in same time units.
    interval = slip interval for characteristic and random_slip in years, use 1 for creep and <= 0 for random time
    tmax = total time in years
    total_slip = total slip in meters. calculated based on tmax and slip rate if not entered.
    slip_rate = slip speed in meters/year. calculated from tmax and total_slip if not entered.
    either total_slip or slip_rate is required.
    '''
    # seed random generator
    if seed:
        np.random.seed(seed)

    # first make sure there are numbers entered
    if total_slip==slip_rate==0:
        raise Warning('slip is zero. must enter value for total_slip or slip_rate')

    # calculate slip_speed based on inputs
    if slip_rate>0:
        slip_rate = slip_rate
        print('slip rate is '+str(np.round(slip_rate,5))+' m/yr')
    else:
        slip_rate = float(total_slip) / tmax
        print('slip rate is '+str(np.round(slip_rate,5))+' m/yr')


    # calculate total_slip based on inputs
    if total_slip>0:
        total_slip = total_slip
        print('total slip is '+str(total_slip)+' meters')
    else:
        total_slip = float(tmax * slip_rate)
        print('total slip is '+str(total_slip)+' meters')

    # checking that rates make sense
    if slip_rate*tmax != total_slip:
        raise Warning("entered values for slip_rate and total_slip don't match!")

    # initialize time & slip arrays
    timesteps = int(tmax/dt)
    slip = np.zeros(timesteps, dtype=float)

    if slip_type=='creep' or interval == 1:
        print('creeping')
        slip_interval = 1        # slip every this many years, 1 for creep
        slip_freq = 1            # slip interval in timesteps
        number_events = int(timesteps/slip_freq)
        slip_per_event = float(total_slip)/number_events
        slip[0::slip_freq] = slip_per_event

    elif slip_type=='characteristic':
        print('characteristic slip')
        slip_interval = interval                        # slip every this many years
        slip_freq = int(slip_interval/dt)            # slip interval in timesteps
        number_events = int(timesteps/slip_freq)
        slip_per_event = float(total_slip)/number_events
        slip[1:-1:slip_freq] = slip_per_event
        if np.sum(slip) > total_slip:
            slip[1] = 0

    elif slip_type=='random_time':
        # this does random event spacing, but magnitude of slip is constant & is not based on time since last event
        print('random time')
        prob_slip = np.random.random_sample(timesteps) #
        prob_slip[prob_slip<=0.95] = 0
        number_events = np.count_nonzero(prob_slip)
        slip_per_event =  float(total_slip)/number_events
        slip[prob_slip>0] = 1
        slip[:] *= slip_per_event

    elif slip_type=='random_slip':
        # this does random slip_per_event based on given interval, sum to total_slip, but not AT ALL based on time since or size of previous eq
        print('random slip')
        slip_interval = interval                        # slip every this many years
        slip_freq = int(slip_interval/dt)            # slip interval in timesteps
        number_events = int(timesteps/slip_freq)
        random_events = np.random.random(number_events)
        random_events *= total_slip/np.sum(random_events)
        slip[1::slip_freq] = random_events
        slip_per_event = random_events

    elif slip_type=='random_random':
        # random timing and random amount slip, sums to total_slip, not AT ALL based on time since or size of previous event
        print('random time & slip')
        prob_slip = np.random.random_sample(timesteps) #
        prob_slip[prob_slip<=0.95] = 0
        number_events = np.count_nonzero(prob_slip)
        random_events = np.random.random(number_events)
        random_events *= total_slip/np.sum(random_events)
        slip[prob_slip>0] = random_events
        slip_per_event = random_events

    elif slip_type=='random_std':
        print('slip based on random sampling of many normal distributions with std=std and centers at interval. slip per event is constant.')
        slip_interval = interval
        number_events = int(tmax/slip_interval) # calc number of earthquakes
        slip_per_event = float(total_slip)/number_events # calc slip_per_event based on

        # define locations (in time) for normal distribution centers
        loc = slip_interval * np.arange(0,number_events,1)
        # keep slip_interval in model_time, not timesteps, so that std numbers are more intuitive (i.e., std 100 = 100 years, not 100/dt years)

        # get normal distriubtion from numpy.random.normal
        out = np.random.normal(loc,std)
        out = out/dt # clean up out - put in timesteps b/c slip is in timesteps
        out[out<0] *= -1 # clean up out - make any negative values positive

        # apply out to slip_regime
        for v in out:
            if int(v) < timesteps:
                slip[int(v)] = slip_per_event

    else: print('WARNING: CHECK SPELLING OF SLIP_TYPE')

    slip_slipped = np.sum(slip)
    eqs_happened = np.sum(slip>0)

    if slip_slipped < total_slip:
        print('WARNING: SLIP IS LESS THAN DESIRED')

    print('total slip slipped is %s meters in %s earthquakes' %(slip_slipped,eqs_happened))

    if plot:
        plt.plot(range(timesteps), slip)
        plt.xlabel('time (ka)')
        plt.ylabel('slip (m)')
        plt.show()

    return slip, slip_per_event


def calcCOV(slip_regime, dt):
    '''Returns COV.
    Function to calculate COV from a slip_regime array of the type used in strike_slip_model.py
    The grid slips (an earthquake!) when slip_regime>0 = intervals[value>0]. the index of intervals lets us get at time
    between earthquakes (recurrence).'''

    # get list of [index,value] for slip_regime
    intervals = list(map(list, enumerate(slip_regime)))

    # swap index and value for each item in intervals
    for item in intervals:
        item[0], item[1] = item[1], item[0]

    # make intervals an array
    intervals = np.asarray(intervals,dtype=float)

    # keep only the earthquakes (intervals[value>0])
    earthquakes = intervals[:][intervals[:,0]>0]

    # keep only interval indexes. get rid of intervals[value] --> we don't need value now that we have intervals in timesteps
    intervals = earthquakes[:,1]

    # initiate recurrence array
    recurrence = np.zeros(len(intervals)-1)

    # calulate recurrence interval (in timesteps) by differencing next and current intervals for all of interval
    for i in range(len(recurrence)):
        recurrence[i] = intervals[i+1] - intervals[i]

    # convert recurrence in timesteps to years
    recurrence = recurrence * dt

    # calculate cov
    cov = np.round(np.std(recurrence) / np.mean(recurrence),3)

    return cov

### Now, let's import packages and set some configurations

In [6]:
import time, os                       # track run time, chdirs, read args from command line
import numpy as np                         # math with numpy
import yaml                                # for reading in a parameter file
import matplotlib.pyplot as plt            # plot with matplotlib
plt.rcParams['axes.facecolor'] =(1,1,1,1)  # white plot backgrounds
plt.rcParams['figure.facecolor']=(1,1,1,0) # clear figure backgrounds
plt.rcParams['xtick.top']=True # plot ticks on top
plt.rcParams['ytick.right']=True # plot ticks on right side
plt.rcParams['ytick.direction']='in' # tick marks face in
plt.rcParams['xtick.direction']='in' # tick marks face in

from slip import calcSlipRegime, calcOFD, calcCOV # my function for making tectonic regimes and ofd profiles

from landlab import RasterModelGrid, imshow_grid # landlab grid components
from landlab.components import LinearDiffuser, DepressionFinderAndRouter
from landlab.components import FlowAccumulator, FastscapeEroder
import pickle as pickle
from landlab.io import read_esri_ascii 

cd Base-code
from slip import calcSlipRegime, calcOFD, calcCOV # my function for making tectonic regimes and ofd profiles
cd ..

SyntaxError: invalid syntax (<ipython-input-6-af9c6b71d7cf>, line 20)

### Next, we'll set various parameters required by the model

In [ ]:
# time parameters
dt = 1 # timestep in years
tmax = 10 # total time in years

# grid parameters
xmax = 1000 # x grid dimension in meters
ymax = 500 # y grid dimension in meters
dxy = 1.0 # grid step in meters

# geomorph parameters
kappa = 0.01 # hillslope diffusivity; bob says this is good value for DV alluvium. see Nash, Hanks papers. m2/yr diffusivity / transport coefficient for soil diffusion
K_sp = 0.003 # stream power for FastscapeEroder
threshold = 0.00005 # threshold for FastscapeEroder
m = 0.5                 # exponent on area in stream power equation
n = 1.                  # exponent on slope in stream power equation

# strike-slip parameters
slip_interval = 1 # how often an earthquake occurs [years]
total_slip = 10. # total slip for entire model time [meters]
slip_type = 'characteristic' # slip type for calc slip regime function. options are characteristic, creep, random_time, random_slip, random_random
control = False # make a model with no tectonics to compare how landscape acts without perturbations
uplift_rate = 0.001 # background relative rock uplift rate [m/yr]
std = 0
seed = 123456
slip_regime = None

# ofd parameters
fault_zone_width = 0.000001 # define width of fault zone (ofd zone) in meters. use 0.000001 for none b/c otherwise get divide by zero error. (distance from fault on either side) [is this half or total? --> total, but not actually used. only used to get ystar = fzw/5]
y_star = fault_zone_width/7. # length scale of OFD decay [meters].

# plotting parameters (grid plotting & initial conditions)
figsize = [10,8] # size of grid plots
shrink = 0.35 # amount of colorbar shrinkage for plots (0-1). 1 = not shrunk. 0 = nonexistent.
limits = [0,20] # elevation limits for grid plots
plots = 100 # how often to save a frame for the movie. 1 = every timestep.

model_time = np.arange(0,tmax,dt) # Set model time parameters

### Before running the model, we need to initialize the model grid and set its boundary conditions. We will use a DEM formatted as an ASCII file as the model's initial condition.

In [ ]:
(grid, z) = read_esri_ascii('Base-code/ztopo_1000x500y1.asc',name='topographic__elevation') 
grid.set_closed_boundaries_at_grid_edges(True, True, True, False) # Boundary conditions
nrows = int(ymax/dxy)
ncols = int(xmax/dxy)

### Let's take a look at our initial condition model domain.

In [ ]:
fig = plt.figure(figsize=figsize)
imshow_grid(grid,z,cmap='terrain',limits=limits,grid_units=['m','m'],shrink=shrink)
plt.axhline(fault_loc,0,xmax,color='k',linestyle='--',linewidth = 0.5)
plt.title('Topography after 0 years')
plt.text(xmax-110,-80, 'total slip: 0 m')
plt.show()

### Now, we need to initiallize various tectonics-related parameters and structures that will allow us to simulate strike-slip faulting and track displacement along the fault.

In [ ]:
# Set tectonic parameters
fault_loc = int(ymax / 2.) # EDITED LINE; Remove 'np.'
# row of nodes that are main fault trace
fault_nodes = np.where(grid.node_y==fault_loc)[0]
# make a slip regime and slip_per_event
slip_regime, slip_per_event = calcSlipRegime(tmax, dt, slip_interval,
                                             std=std, total_slip=total_slip,
                                             slip_type=slip_type, seed=seed)

# calculate COV of slip_regime
cov = calcCOV(slip_regime,dt)

# calculate cumulative_slip
cumulative_slip = np.zeros((len(model_time)),dtype=float)
for i in range(len(model_time)-1):
    if slip_regime[i] > 0:
        cumulative_slip[i] += slip_regime[i]
    cumulative_slip[i+1] = cumulative_slip[i]
    
max_slip = slip_per_event # [meters]

# calculate ofd slip profile, this is length(nrows)
ofd_profile = calcOFD(fault_zone_width,max_slip,fault_loc,y_star,dxy,ymax,nrows,plot=False)

###### SET UP TO TRACK DISPLACEMENT #############
# because the grid is discretized into pixels, we need to count how much deformation has occurred over an earthquake
# and move a pixel after the accumulated deformation is larger than than the pixel length
accum_disp = np.zeros(nrows) # start with no accumulated displacement
accum_disp_total = np.zeros(shape=(nrows,ncols)) # also track total accumulated displacement
displacement = grid.add_zeros('node','accumulated__displacement') # add field to grid to track accumulated displacement

# This is an array for counting how many pixels need to be moved
nshift = np.zeros(nrows,dtype=int)

### Let's see the slip scenario that we just designed for our model.

In [ ]:
# plot slip_regime and cumulative slip
fig = plt.subplots(2,1, figsize=(5,7))
ax1 = plt.subplot(2,1,1)
ax1.plot(model_time, slip_regime,color='steelblue',linewidth=1.0)
ax1.set_ylabel('slip (m)')
ax1.set_xlabel('time (yrs)')
ax1.set_title('slip regime',fontsize=10)
ax2 = plt.subplot(2,1,2)
ax2.plot(model_time,cumulative_slip,color='goldenrod')
ax2.set_ylabel('slip (m)')
ax2.set_xlabel('time (yrs)')
ax2.set_title('cumulative slip',fontsize=10)
ax2.text(0,28,'cov: %s' %cov)
plt.tight_layout()
plt.show()

### Alternatively, the displacement that our designed simulation will perform can be viewed in plan-view.

In [ ]:
# plot ofd profile
fig = plt.subplots(figsize=(5.5,5))
plt.plot(ofd_profile,np.linspace(0,ymax,nrows),color='navy',linewidth=1.2)
plt.axhline(fault_loc,0,1,color='k',linestyle='--',linewidth=0.5)
plt.grid(color='lightgray',linestyle='--')
plt.ylabel('distance (m)')
plt.xlabel('slip (m)')
plt.title('Strike-Slip Displacement Profile')
plt.show()

### Lastly, we will initialize the landlab components used in the model. For hillslope diffusion, we will use the linear_diffuser component. We will route flow and keep track of flow accumulation across the model grid with the D8 flow routing algorithm. Our grid will be hydrologically conditioned with the DepressionFinderandRouter. Fluvial erosion will be governed by the stream power model and performed by the fastscape_eroder component.

In [ ]:
linear_diffuser = LinearDiffuser(grid,linear_diffusivity=kappa) # hillslope diffusion

flow_router = FlowAccumulator(grid, flow_director='D8')

fill = DepressionFinderAndRouter(grid)

fastscape_eroder = FastscapeEroder(grid,K_sp=K_sp, m_sp=m, n_sp=n, threshold_sp=threshold) # stream power erosion

### Initialize a few variables related to model time

In [ ]:
# set start time for keeping track how long a run takes
start_time = time.time()

# calculate number of iterations based on total time
iterations = len(model_time)

# starting an eq counter for the random tectonic regimes
# b/c max_slip is same length as number eq,
# so need to access the right value of max_slip (i.e., the max_slip for that eq)
eq_counter = 0

### Finally, it's time to run the model's main loop

In [ ]:
for i in range(iterations):

    # if this is a time when tectonics happens, do tectonics
    if slip_regime[i] > 0:

        # Take the landlab grid elevations and reshape into a box nrows x ncols
        z_reshape = np.reshape(z[:],[nrows,ncols])

        # Calculate the offset that has accumulated for this time/event
        # this is used to cound how much to shift the grid for this time/event
        # after slip happens, the amount slipped is subtracted from accum_disp
        if len(ofd_profile) == nrows: 
            accum_disp += ofd_profile
            for j in range(ncols): accum_disp_total[:,j]+= ofd_profile

        else:
            accum_disp += ofd_profile[eq_counter,:]
            for j in range(ncols): accum_disp_total[:,j]+= ofd_profile[eq_counter,:]


        # keep track of total accumulated displacement
        accum_disp_total_reshape = np.reshape(accum_disp_total,nrows*ncols)

        # save total accumulated displacement in a field on the grid
        displacement[:] = accum_disp_total_reshape

        # count number of pixels to be moved this time/event
        nshift[:] = np.floor(accum_disp/dxy)

        # now scan up the landscape row by row looking for offset
        for r in range(nrows): # change xrange to range for Py3

            # check if the accumulated offset for a row is larger than a pixel
            if accum_disp[r] >= dxy or accum_disp[r] <= -dxy:

                # move the row over by the number of pixels of accumulated offset
                z_reshape[r,:] = np.roll(z_reshape[r,:],nshift[r])

                # subtract the offset pixels from the displacement
                accum_disp[r] -= dxy * nshift[r]

        # Reshape the elevation box into an array for landlab
        z_new = np.reshape(z_reshape, nrows*ncols)

        # feed new z back into landlab
        z[:] = z_new

        # move the eq_counter ahead
        eq_counter += 1

    # now do the landscape evolution stuff
    # diffuse landscape via soil diffusion
    linear_diffuser.run_one_step(dt)
    # calculate flow routes across new landscape
    flow_router.run_one_step()
    # erode landscape based on routed flow and stream incision
    fastscape_eroder.run_one_step(dt)
    # uplift by background uplift rate
    z[:] += uplift_rate * dt
    # make sure bottom row stays at 0 elevation
    z[grid.node_y==0] = 0

### Let's see how our final time step looks

In [ ]:
fig = plt.figure(figsize=figsize)
imshow_grid(grid,z,cmap='terrain',limits=limits,grid_units=['m','m'],shrink=shrink)
plt.title('Topography after '+str(int((i*dt)+dt))+' years')
plt.text(xmax-110,-80, 'total slip: '+str(np.sum(slip_regime))+' m')
plt.show()

### Next, let's run the drainage area function

In [ ]:
import matplotlib.colors as colors
# DRAINAGE AREA
da = grid["node"]["drainage_area"]
da = np.reshape(da,[nrows,ncols])

# AREA ^ M
Am = da**m

# plot both drainage area and A^m
fig = plt.subplots(2,1,figsize=(10,8))
plt.subplot(2,1,1)
plt.imshow(da,cmap='inferno',origin='lower',
           alpha=1,norm=colors.Normalize(vmax=25000),
           interpolation='none')
plt.colorbar(shrink=.5,label=r'$m^2$')
plt.ylabel('y (m)')
plt.title('Drainage Area')

plt.subplot(2,1,2)
plt.imshow(Am,cmap='inferno',origin='lower',alpha=1,interpolation='none')
plt.colorbar(shrink=.5)
plt.title(r'A$^m$')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.tight_layout()
plt.show()